# Project: Investigation of Soccer Players' Performance

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

I've selected the European Soccer Database to investigate, due to my passion on understanding performance.
This database includes information related to soccer for seasons 2008 to 2016.
I've opened the file using DB Browser, since it's a ".sqlite" file. Within DB Browser I was able to better check the contents of the database (in "Browse Data"), which includes 8 tables, with the following information:
- `Country` (mentions included countries)
- `League` (mentions included leagues and their respective countries)
- `Match` (info about soccer matches, of where and when they occurred, which teams and players played, players' positions in the field, scores, stats related to team performance during the game, the odds of winning, drawing or losing by 10 different providers)
- `Player` (mentions included players, their birthday, height and weight)
- `Player_Attributes` (info about each player in terms of their attributes, such as overall rating, potential, preferred foot, rate of attacking and defensive work, scores related to how good they are in various skills and general attributes, and when these measures were updated)
- `Team` (mentions included teams)
- `Team_Attributes` (info about each team like their scores in various attributes and its respective class, and when these measures were updated)
- `sqlite_sequence` (unrelated to soccer information and unnecessary for this investigation)



### Question(s) for Analysis


Taking into account all information we have and my own curiosity, I've come up with 2 questions:

1. **Is team overall age, height or weight correlated with matches' results?**

This is a type of question very common in recruitment processes, because there is a lot of preconceived ideas about age in certain tasks' performance. I am curious to know if these seem to matter on a team performance, especialy in a sports context, although we are only investigating correlation and not causation.

2. **Do players tend to achieve their perceived potential?**

I want to know how many players were able to reach their first mentioned potential and by how much.

In [1]:
# These are the packages I am expecting to use to answer all questions

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


<a id='wrangling'></a>
## Data Wrangling

In order to answer my questions, I'll only need the tables `Match`, `Player` and `Player_Attributes`. For that reason, I've exported these 3 tables, with only the columns I believe to be necessary, and I'll merge them whenever necessary using python. I've used the following SQL code to do it:

>SELECT *
FROM Match

>SELECT player_api_id, birthday, height, weight
FROM Player

>SELECT player_api_id, date, overall_rating, potential
FROM Player_Attributes

I downloaded the respective CSV files.

### General Properties

Now, let's look at the data to better understand it.

In [2]:
#Load and see Match information
df_match = pd.read_csv("Football_Match.csv")
df_match.head(1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [3]:
#See Match column names (too many columns, not able to use info())
df_match.columns.values

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'aw

In [4]:
#Load and see Player information
df_player = pd.read_csv("Football_Player.csv")
df_player.head(1)

,player_api_id,birthday,height,weight
0,505942,1992-02-29 00:00:00,182.88,187


In [5]:
#See Player columns and their datatypes
df_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player_api_id  11060 non-null  int64  
 1   birthday       11060 non-null  object 
 2   height         11060 non-null  float64
 3   weight         11060 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 345.8+ KB


In [6]:
#Load and see Player_Attributes information
df_playeratt = pd.read_csv("Football_Player_Attributes.csv")
df_playeratt.head(1)

,player_api_id,date,overall_rating,potential
0,505942,2016-02-18 00:00:00,67.0,71.0


In [7]:
#See Player columns and their datatypes
df_playeratt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   player_api_id   183978 non-null  int64  
 1   date            183978 non-null  object 
 2   overall_rating  183142 non-null  float64
 3   potential       183142 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 5.6+ MB


Now I know all columns within the original tables. I won't explore them just yet, since there are too many. I'll first create tables with only the needed columns and then do Data Exploration Analysis within those.


### Data Cleaning

First, let's create the dataframe we need for the first question.
We need it to have the following columns:
- `match_api_id`
- `home_team`
- `away_team`
- `Overall_Age`
- `Overall_Height`
- `Overall_Weight`
- `Win_Rate`
- `Draw_Rate`
- `Loss_Rate`

We'll go one by one.

In [8]:
# I know I'll have to change datatype of birthday in Player, but I need to know the datetype of date in Match too
df_match["date"].dtypes

dtype('O')

In [9]:
# We need to change datatype of birthdate in Player and in date of Match
df_player["birthday"] = pd.to_datetime(df_player["birthday"])
df_match["date"] = pd.to_datetime(df_match["date"])

In [10]:
df_player.dtypes

player_api_id             int64
birthday         datetime64[ns]
height                  float64
weight                    int64
dtype: object

In [11]:
# Now we need to know the average age. We'll get the average age of the team per match, first.
#This list will help
home_team=['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5','home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11']
away_team=['away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6','away_player_7','away_player_8','away_player_9','away_player_10','away_player_11']

# Let's drop missing values on players id before we go on
df_match.dropna(subset=home_team, inplace=True)
df_match.dropna(subset=away_team, inplace=True)

In [12]:
# Now we'll focus on preparing our table for question1
match_columns1 = ["date","match_api_id","home_team_api_id","away_team_api_id"]
match_columns_final = match_columns1 + home_team + away_team
df_question1 = df_match.loc[:,match_columns_final]
df_question1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21374 entries, 145 to 25978
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              21374 non-null  datetime64[ns]
 1   match_api_id      21374 non-null  int64         
 2   home_team_api_id  21374 non-null  int64         
 3   away_team_api_id  21374 non-null  int64         
 4   home_player_1     21374 non-null  float64       
 5   home_player_2     21374 non-null  float64       
 6   home_player_3     21374 non-null  float64       
 7   home_player_4     21374 non-null  float64       
 8   home_player_5     21374 non-null  float64       
 9   home_player_6     21374 non-null  float64       
 10  home_player_7     21374 non-null  float64       
 11  home_player_8     21374 non-null  float64       
 12  home_player_9     21374 non-null  float64       
 13  home_player_10    21374 non-null  float64       
 14  home_player_11    21

In [13]:
# Now we're going to get the birthday information for each home player
for number in range(11):
    df_question1 = df_question1.merge(df_player, left_on="home_player_"+str(number+1), right_on="player_api_id", how="left", suffixes=(None,"_h"+str(number+1)))
    df_question1.drop("player_api_id", axis=1, inplace=True)
    
df_question1.rename(columns={"birthday":"birthday_h1","height":"height_h1","weight":"weight_h1"}, inplace=True)

In [14]:
# And now for each away player
for number in range(11):
    df_question1 = df_question1.merge(df_player, left_on="away_player_"+str(number+1), right_on="player_api_id", how="left", suffixes=(None,"_a"+str(number+1)))
    df_question1.drop("player_api_id", axis=1, inplace=True)
    
df_question1.rename(columns={"birthday":"birthday_a1","height":"height_a1","weight":"weight_a1"}, inplace=True)

In [15]:
df_copy = df_question1.copy()

In [16]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21374 entries, 0 to 21373
Data columns (total 92 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              21374 non-null  datetime64[ns]
 1   match_api_id      21374 non-null  int64         
 2   home_team_api_id  21374 non-null  int64         
 3   away_team_api_id  21374 non-null  int64         
 4   home_player_1     21374 non-null  float64       
 5   home_player_2     21374 non-null  float64       
 6   home_player_3     21374 non-null  float64       
 7   home_player_4     21374 non-null  float64       
 8   home_player_5     21374 non-null  float64       
 9   home_player_6     21374 non-null  float64       
 10  home_player_7     21374 non-null  float64       
 11  home_player_8     21374 non-null  float64       
 12  home_player_9     21374 non-null  float64       
 13  home_player_10    21374 non-null  float64       
 14  home_player_11    2137

In [17]:
# TENTATIVE: NOT WORKING
for number in range(11):
    df_copy["birthday_h"+str(number+1)] = df_copy["date"].dt.year - df_copy["birthday_h"+str(number+1)].dt.year - ((df_copy["date"].dt.month, df_copy["date"].dt.day) < (df_copy["birthday_h"+str(number+1)].dt.month,df_copy["birthday_h"+str(number+1)].dt.day))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
# NOT WORKING
# Now let's convert all datetime data with the respective player age. We'll first create the function that will do it
def know_age(birthday):
    return df_copy["date"].year - birthday.year -((df_copy["date"].month, df_copy["date"].day) < (birthday.month,birthday.day))

In [19]:
# NOT WORKING
# Let's apply it to home players
for number in range(11):
    df_copy["birthday_h"+str(number+1)] = df_copy.apply(lambda x: know_age(x))

AttributeError: 'Series' object has no attribute 'year'

Ver isto mais tarde: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. **Compute statistics** and **create visualizations** with the goal of addressing the research questions that you posed in the Introduction section. You should compute the relevant statistics throughout the analysis when an inference is made about the data. Note that at least two or more kinds of plots should be created as part of the exploration, and you must  compare and show trends in the varied visualizations. 



> **Tip**: - Investigate the stated question(s) from multiple angles. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables. You should explore at least three variables in relation to the primary question. This can be an exploratory relationship between three variables of interest, or looking at how two independent variables relate to a single dependent variable of interest. Lastly, you  should perform both single-variable (1d) and multiple-variable (2d) explorations.


### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed in relation to the question(s) provided at the beginning of the analysis. Summarize the results accurately, and point out where additional research can be done or where additional information could be useful.


> **Tip**: If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

### Limitations
> **Tip**: Make sure that you are clear with regards to the limitations of your exploration. You should have at least 1 limitation explained clearly. 

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> **Tip**: Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> **Tip**: Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> **Tip**: Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])